In [ ]:
import numpy as np
from PIL import Image
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

transform = transforms.Compose([transforms.ToTensor()])
A = []
file = open("train_triplets.txt", "r")
data = file.readline().split()
data = file.readline().split()
data = file.readline().split()
data = file.readline().split()
A.append(
    transform(
        Image.open(f"food/{data[0]}.jpg").resize((64, 32)).convert("RGB")
    )
)

In [ ]:
display(A[0].shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# show images
imshow(A[0])
# print labels

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(13 * 13 * 16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x)  # flatten all dimensions
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
convert_tensor = transforms.Compose([transforms.ToTensor()])

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    file = open("train_triplets.txt", "r")
    i = 0
    for line in file:
        i += 1
        data = line.split()
        A = convert_tensor(
            Image.open(f"food/{data[0]}.jpg").resize((64, 32)).convert("RGB")
        )
        B = convert_tensor(
            Image.open(f"food/{data[1]}.jpg").resize((64, 32)).convert("RGB")
        )
        C = convert_tensor(
            Image.open(f"food/{data[2]}.jpg").resize((64, 32)).convert("RGB")
        )
        # get the inputs; data is a list of [inputs, labels]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize

        outputs = net(torch.cat((A, B), 1))
        loss = criterion(outputs, torch.tensor([1.0]))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(torch.cat((A, C), 1))
        loss = criterion(outputs, torch.tensor([0.0]))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        if i % 2000 == 1999:  # print every 2000 files
            print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
            running_loss = 0.0

print("Finished Training")